### Restart and run all cells

#### csv files: sell-alert

In [1]:
import calendar
import pandas as pd
import numpy as np
from datetime import date, timedelta
from sqlalchemy import create_engine
from math import ceil, floor, trunc

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
con_lite = engine.connect()

engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
today

datetime.date(2022, 3, 24)

In [2]:
def number_of_spread(x):
    maxp = x[1]
    minp = x[10]
    div = 0.0
    gap = 0.0
    spd = 0
    spd1 = 0
    spd2 = 0
    if (maxp > minp): 
        mult = 1
    else:
    	tmpp = minp
    	minp = maxp
    	maxp = tmpp
    	mult = -1
        
    gap = round(maxp - minp,2)   
    
    if (maxp > 400.00):
        div = 2.0
        boundary = div * 200
        if ((minp < boundary)):
            spd1 = trunc((boundary - minp) / 1.00)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div) 
    elif (maxp > 200.00):
        div = 1.0
        boundary = div * 200
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.50)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div) 
    elif (maxp > 100.00):
        div = 0.50
        boundary = div * 200
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.25)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div) 
    elif (maxp > 25.00):
        div = 0.25
        boundary = div * 100
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.10)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            spd = trunc(gap / div)
    elif (maxp > 10.00):
        div = 0.10
        boundary = div * 100
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.05)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2
        else:
            div = 0.10            
            spd = trunc(gap / div)
    elif (maxp > 5.00):
        div = 0.05
        boundary = div * 100
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.02)
            spd2 = trunc(round(((maxp - boundary) / div),2))
            spd = spd1 + spd2           
        else:
            div = 0.05            
            spd = ceil(gap / div)
    elif (maxp > 2.00):
        div = 0.02
        boundary = div * 100
        if (minp < boundary):
            spd1 = trunc((boundary - minp) / 0.01)
            spd2 = trunc((maxp - boundary) / div)
            spd = spd1 + spd2
        else:          
            spd = (gap / div)
    else:
        div = 0.01
        boundary = div * 100        
        spd = trunc(gap / div) 
        
    return spd * mult

In [3]:
format_dict = {
               'fm_price':'{:.2f}','to_price':'{:.2f}',    
               'max_price':'{:.2f}','min_price':'{:.2f}',
               'target':'{:.2f}','max':'{:.2f}','max52':'{:.2f}','css_price':'{:.2f}','amount':'{:,.2f}',  
               'unit_cost':'{:.2f}','cost_amt':'{:,.2f}',
               'dividend':'{:.2f}','dividend_amount':'{:,.2f}',    
               'cost_amt':'{:,.2f}','div_amt':'{:,.2f}','cost_pct':'{:.2f}', 
               'buy_target':'{:.2f}','sell_target':'{:.2f}',
               'volume':'{:,.2f}','beta':'{:.2f}','cost':'{:,.2f}',
               'qty':'{:,}','available_qty':'{:,}','shares':'{:,}'}
pd.set_option("display.max_rows", None)

In [4]:
sql = '''
SELECT T.name,sell_target AS target, sh, T.qty,T.available_qty,cost,
A.fm_date, A.to_date, days, fm_price, to_price, ttl_spread AS ttl, 
A.max_price AS max, T.max_price AS max52, consensus_price AS css_price,
sell_target * T.qty AS amount, T.status
FROM stocks T
JOIN sales A
ON T.name = A.name
LEFT JOIN consensus C
ON T.name = C.name
WHERE T.status IN ('I','S')
AND to_date = '%s'
'''
# AND to_price >= fm_price
sql = sql % today
print(sql)

df_stock = pd.read_sql(sql, con_lite)
df_stock.sort_values(['name'],ascending=[True]).style.format(format_dict)


SELECT T.name,sell_target AS target, sh, T.qty,T.available_qty,cost,
A.fm_date, A.to_date, days, fm_price, to_price, ttl_spread AS ttl, 
A.max_price AS max, T.max_price AS max52, consensus_price AS css_price,
sell_target * T.qty AS amount, T.status
FROM stocks T
JOIN sales A
ON T.name = A.name
LEFT JOIN consensus C
ON T.name = C.name
WHERE T.status IN ('I','S')
AND to_date = '2022-03-24'



,name,target,sh,qty,available_qty,cost,fm_date,to_date,days,fm_price,to_price,ttl,max,max52,css_price,amount,status
11,AIMIRT,13.10,0,"10,000","10,000",13.00,2022-03-23,2022-03-24,1,12.70,12.80,1,12.90,13.40,15.20,"131,000.00",I
14,ASP,3.82,0,"30,000","30,000",3.80,2022-03-23,2022-03-24,1,3.48,3.46,-1,3.48,4.24,4.65,"114,600.00",I
12,BCH,22.40,0,"5,000","15,000",21.30,2022-03-23,2022-03-24,1,21.30,21.40,1,21.60,26.75,24.20,"112,000.00",I
4,CPNCG,12.50,0,"20,000","20,000",12.40,2022-03-21,2022-03-24,3,12.40,12.20,-2,12.40,12.80,nan,"250,000.00",I
10,DOHOME,23.80,0,"4,000","12,000",23.70,2022-03-22,2022-03-24,2,23.00,21.30,-17,22.30,25.62,25.24,"95,200.00",I
8,HREIT,9.00,12,"40,100","40,100",8.95,2022-03-22,2022-03-24,2,8.70,8.65,-1,8.70,9.30,9.10,"360,900.00",I
7,IP,20.30,0,"2,500","7,500",20.20,2022-03-22,2022-03-24,2,19.10,18.90,-2,19.20,24.47,25.00,"50,750.00",I
0,JASIF,11.30,0,"10,000","110,000",10.10,2022-03-08,2022-03-24,12,10.40,11.00,6,11.00,11.30,11.32,"113,000.00",I
2,KCE,68.00,0,"1,000","13,000",73.25,2022-03-21,2022-03-24,3,62.75,65.25,10,65.25,95.00,74.99,"68,000.00",I
3,PTT,41.25,0,"3,000","9,000",38.50,2022-03-21,2022-03-24,3,39.00,39.50,2,39.50,42.00,49.04,"123,750.00",I


In [5]:
my_array = df_stock.to_numpy()
my_array

array([['JASIF', 11.3, 0, 10000, 110000, 10.1, '2022-03-08',
        '2022-03-24', 12, 10.4, 11.0, 6, 11.0, 11.3, 11.32, 113000.0,
        'I'],
       ['RCL', 52.0, 0, 3000, 12000, 47.25, '2022-03-21', '2022-03-24',
        3, 51.25, 45.5, -23, 47.25, 67.75, 56.0, 156000.0, 'I'],
       ['KCE', 68.0, 0, 1000, 13000, 73.25, '2022-03-21', '2022-03-24',
        3, 62.75, 65.25, 10, 65.25, 95.0, 74.99, 68000.0, 'I'],
       ['PTT', 41.25, 0, 3000, 9000, 38.5, '2022-03-21', '2022-03-24', 3,
        39.0, 39.5, 2, 39.5, 42.0, 49.04, 123750.0, 'I'],
       ['CPNCG', 12.5, 0, 20000, 20000, 12.4, '2022-03-21', '2022-03-24',
        3, 12.4, 12.2, -2, 12.4, 12.8, nan, 250000.0, 'I'],
       ['SINGER', 52.75, 12, 1500, 1500, 42.25, '2022-03-22',
        '2022-03-24', 2, 49.75, 51.75, 8, 52.0, 55.0, 58.71, 79125.0,
        'I'],
       ['TPIPL', 1.71, 0, 60000, 60000, 1.7, '2022-03-22', '2022-03-24',
        2, 1.67, 1.64, -3, 1.66, 2.22, 2.43, 102600.0, 'I'],
       ['IP', 20.3, 0, 2500, 7500, 2

In [6]:
tmp = (np.apply_along_axis(number_of_spread, axis=1, arr=my_array))
tmp

array([ 2, 26, 11,  7,  2,  4,  7, 13,  7, 11, 25,  2, 10,  8, 18])

In [7]:
df_stock['spd'] = tmp.tolist()
df_stock['spd'] = df_stock['spd'].astype(int)
df_stock.style.format(format_dict)

,name,target,sh,qty,available_qty,cost,fm_date,to_date,days,fm_price,to_price,ttl,max,max52,css_price,amount,status,spd
0,JASIF,11.30,0,"10,000","110,000",10.10,2022-03-08,2022-03-24,12,10.40,11.00,6,11.00,11.30,11.32,"113,000.00",I,2
1,RCL,52.00,0,"3,000","12,000",47.25,2022-03-21,2022-03-24,3,51.25,45.50,-23,47.25,67.75,56.00,"156,000.00",I,26
2,KCE,68.00,0,"1,000","13,000",73.25,2022-03-21,2022-03-24,3,62.75,65.25,10,65.25,95.00,74.99,"68,000.00",I,11
3,PTT,41.25,0,"3,000","9,000",38.50,2022-03-21,2022-03-24,3,39.00,39.50,2,39.50,42.00,49.04,"123,750.00",I,7
4,CPNCG,12.50,0,"20,000","20,000",12.40,2022-03-21,2022-03-24,3,12.40,12.20,-2,12.40,12.80,nan,"250,000.00",I,2
5,SINGER,52.75,12,"1,500","1,500",42.25,2022-03-22,2022-03-24,2,49.75,51.75,8,52.00,55.00,58.71,"79,125.00",I,4
6,TPIPL,1.71,0,"60,000","60,000",1.70,2022-03-22,2022-03-24,2,1.67,1.64,-3,1.66,2.22,2.43,"102,600.00",I,7
7,IP,20.30,0,"2,500","7,500",20.20,2022-03-22,2022-03-24,2,19.10,18.90,-2,19.20,24.47,25.00,"50,750.00",I,13
8,HREIT,9.00,12,"40,100","40,100",8.95,2022-03-22,2022-03-24,2,8.70,8.65,-1,8.70,9.30,9.10,"360,900.00",I,7
9,WHART,12.40,0,"20,000","20,000",12.30,2022-03-22,2022-03-24,2,11.50,11.20,-3,11.30,14.08,14.90,"248,000.00",I,11


In [8]:
sql = '''
SELECT name, volbuy AS shares, price AS unit_cost, volbuy * price AS cost_amt, dividend,
volbuy * dividend AS div_amt, (volbuy * dividend)/(volbuy * price) * 100 AS cost_pct, period
FROM buy
WHERE period IN ("1","2","3","4")
'''
df_buy = pd.read_sql(sql, const)
df_buy['shares'] = df_buy.shares.astype(int)
df_buy.sort_values(['name'],ascending=[True]).style.format(format_dict)

,name,shares,unit_cost,cost_amt,dividend,div_amt,cost_pct,period
22,AIMIRT,"10,000",13.00,"130,000.00",0.88,"8,822.00",6.79,4
12,ASP,"30,000",3.80,"114,000.00",0.40,"12,000.00",10.53,4
8,BCH,"15,000",21.30,"319,500.00",1.20,"18,000.00",5.63,4
23,CPNCG,"20,000",12.40,"248,000.00",0.96,"19,174.00",7.73,2
4,DIF,"50,000",14.70,"735,000.00",1.04,"52,200.00",7.10,2
10,DOHOME,"12,000",23.70,"284,400.00",0.00,0.00,0.00,4
15,HREIT,"40,100",8.95,"358,895.00",0.70,"27,877.52",7.77,2
28,IP,"7,500",20.20,"151,500.00",0.00,0.00,0.00,3
11,JASIF,"110,000",10.10,"1,111,000.00",0.95,"104,500.00",9.41,2
2,KCE,"13,000",73.25,"952,250.00",1.60,"20,800.00",2.18,1


In [9]:
df_merge = pd.merge(df_stock, df_buy, on=['name'], how='inner')
cols = 'name spd target sh qty available_qty cost fm_price to_price ttl max max52 css_price status period amount'.split()
df_merge[cols].style.format(format_dict)

,name,spd,target,sh,qty,available_qty,cost,fm_price,to_price,ttl,max,max52,css_price,status,period,amount
0,JASIF,2,11.30,0,"10,000","110,000",10.10,10.40,11.00,6,11.00,11.30,11.32,I,2,"113,000.00"
1,RCL,26,52.00,0,"3,000","12,000",47.25,51.25,45.50,-23,47.25,67.75,56.00,I,3,"156,000.00"
2,KCE,11,68.00,0,"1,000","13,000",73.25,62.75,65.25,10,65.25,95.00,74.99,I,1,"68,000.00"
3,PTT,7,41.25,0,"3,000","9,000",38.50,39.00,39.50,2,39.50,42.00,49.04,I,4,"123,750.00"
4,CPNCG,2,12.50,0,"20,000","20,000",12.40,12.40,12.20,-2,12.40,12.80,nan,I,2,"250,000.00"
5,SINGER,4,52.75,12,"1,500","1,500",42.25,49.75,51.75,8,52.00,55.00,58.71,I,3,"79,125.00"
6,TPIPL,7,1.71,0,"60,000","60,000",1.70,1.67,1.64,-3,1.66,2.22,2.43,I,4,"102,600.00"
7,IP,13,20.30,0,"2,500","7,500",20.20,19.10,18.90,-2,19.20,24.47,25.00,I,3,"50,750.00"
8,HREIT,7,9.00,12,"40,100","40,100",8.95,8.70,8.65,-1,8.70,9.30,9.10,I,2,"360,900.00"
9,WHART,11,12.40,0,"20,000","20,000",12.30,11.50,11.20,-3,11.30,14.08,14.90,I,4,"248,000.00"


In [10]:
sell_alert = df_merge[cols]
sell_alert.set_index(['name'],inplace=True)
sell_alert.sort_values(['spd'],ascending=[True])

,spd,target,sh,qty,available_qty,cost,fm_price,to_price,ttl,max,max52,css_price,status,period,amount
name,,,,,,,,,,,,,,,
JASIF,2,11.30,0,10000,110000,10.10,10.40,11.00,6,11.00,11.30,11.32,I,2,113000.0
CPNCG,2,12.50,0,20000,20000,12.40,12.40,12.20,-2,12.40,12.80,NaN,I,2,250000.0
AIMIRT,2,13.10,0,10000,10000,13.00,12.70,12.80,1,12.90,13.40,15.20,I,4,131000.0
SINGER,4,52.75,12,1500,1500,42.25,49.75,51.75,8,52.00,55.00,58.71,I,3,79125.0
PTT,7,41.25,0,3000,9000,38.50,39.00,39.50,2,39.50,42.00,49.04,I,4,123750.0
TPIPL,7,1.71,0,60000,60000,1.70,1.67,1.64,-3,1.66,2.22,2.43,I,4,102600.0
HREIT,7,9.00,12,40100,40100,8.95,8.70,8.65,-1,8.70,9.30,9.10,I,2,360900.0
RATCH,8,46.00,3,6000,6000,45.75,43.50,44.00,2,44.25,53.75,53.97,I,3,276000.0
BCH,10,22.40,0,5000,15000,21.30,21.30,21.40,1,21.60,26.75,24.20,I,4,112000.0


In [11]:
file_name = 'sell-alert.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

sell_alert.sort_values(['spd'],ascending=[True]).to_csv(output_file)
sell_alert.sort_values(['spd'],ascending=[True]).to_csv(data_file)
sell_alert.sort_values(['spd'],ascending=[True]).to_csv(box_file)
sell_alert.sort_values(['spd'],ascending=[True]).to_csv(one_file)